<a href="https://colab.research.google.com/github/TomerShimshi/DeepLearning-course-Final-Project-Brats-challenge/blob/main/OurProjectipynb_resnet10_seg_4_%2Bunet_with_grad_no_outputs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import argparse
!git clone https://github.com/Tencent/MedicalNet
%load MedicalNet/setting.py


load all relevent libs

In [ ]:


import math
import os
import random
import matplotlib.pyplot as plt
import torch, fastai, sys
sys.path.insert(0, './exp')
from sklearn.preprocessing import Binarizer
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

!pip install medpy
from medpy import metric

import numpy as np
from torch.utils.data import Dataset
import nibabel
from scipy import ndimage
import h5py
from torchvision import transforms

import torch, fastai, sys, os
!pip install antspyx
from fastai.vision import *
#import ants
import pathlib
#from ants.core.ants_image import ANTsImage
!pip install jupyterthemes
from jupyterthemes import jtplot
sys.path.insert(0, './exp')
jtplot.style(theme='gruvboxd')

mount to drive

In [ ]:

from google.colab import drive
drive.mount('/content/drive')
path = Path('/content/drive/My Drive/Deep Learning Final Project/MICCAI_BraTS_2019_Data_Training')
print(path)


patch added the settings.py file

In [ ]:
#@title argparser
'''
Configs for training & testing
Written by Whalechen
'''

import argparse

#parser = argparse.ArgumentParser()

def parse_opts():
    
    parser = argparse.ArgumentParser()
    
    parser.add_argument(
        '--data_root',
        default='/content/drive/My Drive/Deep Learning Final Project/MICCAI_BraTS_2019_Data_Training',
        type=str,
        help='Root directory path of data')
    parser.add_argument(
        '--input_patch_size',
        default='120, 136, 120',
        type=str,
        help='Root directory path of data')
    parser.add_argument(
        '--orig_patch_size',
        default='120, 136, 120',
        type=str,
        help='list of HWD of input')
    parser.add_argument(
        '--img_list',
        default='/content/drive/My Drive/Deep Learning Final Project/MICCAI_BraTS_2019_Data_Training/test.txt',
        type=str,
        help='Path for image list file')
    parser.add_argument(
        '--img_name',
        default='Null',
        type=str,
        help='Path for image current img')
    parser.add_argument('--iters', type=str,  default='11000,10000', help='checkpoint iteration(s)')
    parser.add_argument(
        '--n_seg_classes',
        default=3,
        type=int,
        help="Number of segmentation classes"
    )
    #print('finished 1 method')
    parser.add_argument(
        '--learning_rate',  # set to 0.001 when finetune
        default=0.001,
        type=float,
        help=
        'Initial learning rate (divided by 10 while training by lr scheduler)')
    
    parser.add_argument(
        '--num_workers',
        default=4,
        type=int,
        help='Number of jobs')
    parser.add_argument(
        '--batch_size', default=1, type=int, help='Batch Size')
    parser.add_argument(
        '--phase', default='train', type=str, help='Phase of train or test')
    parser.add_argument(
        '--save_intervals',
        default=10,
        type=int,
        help='Interation for saving model')
    parser.add_argument(
        '--n_epochs',
        default=200,
        type=int,
        help='Number of total epochs to run')
    parser.add_argument(
        '--input_D',
    default=56,
        type=int,
        help='Input size of depth')
    parser.add_argument(
        '--input_H',
        default=448,
        type=int,
        help='Input size of height')
    parser.add_argument(
        '--input_W',
        default=448,
        type=int,
        help='Input size of width')
    parser.add_argument(
        '--resume_path',
        default='',
        type=str,
        help=
        'Path for resume model.'
    )
    
    #print('finished 2 method')
    parser.add_argument(
        '--bce_weight',
        default='[0., 3, 1, 1.75]',
        type=list,
        help=
        'not sure, somthing for the loss func.'
    )
    parser.add_argument(
        '--new_layer_names',
        #default=['upsample1', 'cmp_layer3', 'upsample2', 'cmp_layer2', 'upsample3', 'cmp_layer1', 'upsample4', 'cmp_conv1', 'conv_seg'],
        default=['conv_seg'],
        type=list,
        help='New layer except for backbone')
    parser.add_argument(
        '--no_cuda', action='store_true', help='If true, cuda is not used.')
    parser.set_defaults(no_cuda=False)
    parser.add_argument(
        '--gpu_id',
        nargs='+',
        type=int,              
        help='Gpu id lists')
    parser.add_argument(
        '--model',
        default='resnet',
        type=str,
        help='(resnet | preresnet | wideresnet | resnext | densenet | ')
    parser.add_argument(
        '--model_depth',
        default=50,
        type=int,
        help='Depth of resnet (10 | 18 | 34 | 50 | 101)')
    
    parser.add_argument(
        '--resnet_shortcut',
        default='B',
        type=str,
        help='Shortcut type of resnet (A | B)')
    
    #print('finished 3 method')
    
    parser.add_argument(
        '--manual_seed', default=1, type=int, help='Manually set random seed')
    
    #print('finished 4 method')

    #!python3 parsing.py 5 "arg2"      

    #parser.add_argument(
     #   '--ci_test', action='store_true', help='If true, ci testing is used.')
    parser.add_argument(
        '--ci_test', action='store_true', help='If true, ci testing is used.')
    args = parser.parse_args(args=[])
    #args = parser.parse_args()
    
    #print('finished 5 method')
    !python3 parsing.py 5
    #for avi
    #args.save_folder = "/content/drive/My Drive/Deep Learning Final Project/trails/models/{}_{}".format(args.model, args.model_depth)
    #for shimshi
    args.save_folder = "/content/drive/My Drive/Deep Learning Final Project/trails/models/{}_{}".format( args.model,  args.model_depth )
    print('finished last method')
    return args

sets =parse_opts()  

print('passed sets argparaser')

sets.img_list = path/'test.txt' #'./toy_data/test_ci.txt' 
sets.n_epochs = 1
sets.no_cuda = True
sets.data_root = path # './toy_data'
sets.pretrain_path =  path/'pretrain/resnet_50.pth'
sets.num_workers = 0
sets.model_depth = 50
sets.resnet_shortcut = 'A'
sets.input_D = 14
sets.input_H = 28
sets.input_W = 28

print('finished sets')

#BrainS18Dataset(sets.data_root, sets.img_list, sets)
#BrainS18Dataset(path,loadtxt_str(path/'test.txt'))

trying a new data loader

In [ ]:


from tqdm import tqdm


#----- this is the new brats map label

def brats_map_label(labels, num_classes):
    if type(labels) == torch.Tensor and torch.cuda.is_available():
        labels_nhot = torch.zeros((num_classes,) + labels.shape, device='cuda')
    else:
        labels_nhot = np.zeros((num_classes,) + labels.shape, dtype=int)
    #https://www.med.upenn.edu/cbica/brats2019/data.html and https://arxiv.org/pdf/1811.02629.pdf here is the mapping  page 10
    labels_nhot[0, labels==0] = 1
    labels_nhot[1,(labels==2)|(labels==3) |(labels==1)] = 1     #WT: Segmenting the whole tumor extent (Union of all labels)   # P(WT) = P(1)+P(2)+P(3)      
    labels_nhot[2,labels==3] = 1    # P(ET) = P(3)
    labels_nhot[3,(labels==1)|(labels==3)] = 1  #TC: Segmenting the gross tumor core outline # P(TC) = P(1)+P(3) 
    return labels_nhot




def label_stats(root,img_list):
    with open(img_list, 'r') as f:
            img_dirs = [line.strip() for line in f]
    
    cls_total_counts = np.zeros(4, dtype=int)
        
    for img_dir in tqdm(img_dirs):
        
        img_path= str(root)+'/'+str(img_dir)+'/'+img_dir[3:] +  ".h5"
        
        image_h5 = h5py.File(img_path)
        labels = np.array(image_h5['label']).astype(int)
        labels -= (labels == 4)
        labels_nhot = brats_map_label(labels, 4)
        cls_counts = labels_nhot.reshape(4, -1).sum(axis=1)
        print('cls_counts={}'.format(cls_counts))
        cls_total_counts += cls_counts

        tempL = np.nonzero(labels_nhot[2] == 1)
        # Find the boundary of non-zero labels
        minx, maxx = np.min(tempL[0]), np.max(tempL[0])
        miny, maxy = np.min(tempL[1]), np.max(tempL[1])
        minz, maxz = np.min(tempL[2]), np.max(tempL[2])
        H, W, D = maxx - minx, maxy - miny, maxz - minz
        
        print("%s => %s, %.2f%%" %(labels.shape, (H, W, D), (100*H*W*D/labels.size)))
                
    print(cls_total_counts)
    print(cls_total_counts / cls_total_counts.sum())
    
def covert_h5(root,img_list):
    do_localization = False
    
    if 'val' in str(img_list.stem):
        is_training = False
    else:
        print('training')
        is_training = True
     
    
    with open(img_list, 'r') as f:
            img_dirs = [line.strip() for line in f]
    
    print('img_dirs = {}'.format(img_dirs))
    modalities = [ 'flair', 't1ce', 't1', 't2' ]
    if is_training:
        print('added seg')
        modalities.append('seg')
        
    for img_dir in tqdm(img_dirs):
        image_mods = []
        for mod in modalities:
            
            img_path= str(root)+'/'+str(img_dir)+'/'+img_dir[3:] + "_%s.nii.gz" %mod
            image_obj = nibabel .load(img_path)
            print('imge after load = {}'.format(image_obj))
            image = image_obj.get_fdata()
            image = image.astype(np.float32)
            
            image_mods.append(image)
        
        if is_training:
            # image_mods contains five modalities, including 'seg'.
            # Avoid putting 'seg' into 'image'.
            image_mm = np.stack(image_mods[:-1], axis=0)
            print('imge after np.stack = {}'.format(image_mm))
            MOD, H, W, D = image_shape = image_mm.shape
            # 'seg' => labels
            labels = image_mods[-1].astype(np.uint8)
            print(' labels ={}'.format(labels))
        else:
            # image_mods contains four modalities
            image_mm = np.stack(image_mods, axis=0)
            MOD, H, W, D = image_shape = image_mm.shape
            # Save fake labels
            labels = np.zeros_like(image_mods[0]).astype(np.uint8)
            
        if is_training and do_localization:
            tempL = np.nonzero(labels)
            # Find the boundary of non-zero labels
            minx, maxx = np.min(tempL[1]), np.max(tempL[1])
            miny, maxy = np.min(tempL[2]), np.max(tempL[2])
            minz, maxz = np.min(tempL[3]), np.max(tempL[3])

            # px, py, pz ensure the output image is at least of output_size
            px = max(output_size[0] - (maxx - minx), 0) // 2
            py = max(output_size[1] - (maxy - miny), 0) // 2
            pz = max(output_size[2] - (maxz - minz), 0) // 2
            # randint(10, 20) lets randomly-sized zero margins included in the output image
            minx = max(minx - np.random.randint(10, 20) - px, 0)
            maxx = min(maxx + np.random.randint(10, 20) + px, H)
            miny = max(miny - np.random.randint(10, 20) - py, 0)
            maxy = min(maxy + np.random.randint(10, 20) + py, W)
            minz = max(minz - np.random.randint(5, 10) - pz, 0)
            maxz = min(maxz + np.random.randint(5, 10) + pz, D)
        # Do not crop black margins of validation images.
        else:
            tempL = np.nonzero(image_mm)
            # Find the boundary of non-zero voxels
            minx, maxx = np.min(tempL[1]), np.max(tempL[1])
            miny, maxy = np.min(tempL[2]), np.max(tempL[2])
            minz, maxz = np.min(tempL[3]), np.max(tempL[3])
            
        # On validation data, use image_crop to compute mean and std, and normalize image_mm.
        image_crop = image_mm[:, minx:maxx, miny:maxy, minz:maxz]
        if is_training:
            image_mm = image_crop
            labels = labels[minx:maxx, miny:maxy, minz:maxz]

        nonzero_mask = (image_mm > 0)
        for m in range(MOD):
            image_mod      = image_mm[m, :, :, :]
            image_mod_crop = image_crop[m, :, :, :]
            nonzero_voxels = image_mod_crop[image_mod_crop > 0]
            mean = nonzero_voxels.mean()
            std  = nonzero_voxels.std()
            image_mm[m, :, :, :] = (image_mod - mean) / std
        
        # Set voxels back to 0 if they are 0 before normalization.
        image_mm *= nonzero_mask
        print("\n%s: %s => %s, %s" %(img_dir, image_shape, image_mm.shape, labels.shape))

        #h5_path = os.path.join(root, img_dir, img_dir + ".h5")
        h5_path = str(root)+'/'+str(img_dir)+'/'+img_dir[3:] + ".h5"
        f = h5py.File(h5_path, 'w')
        f.create_dataset('image', data=image_mm, compression="gzip")
        f.create_dataset('label', data=labels,   compression="gzip")
        f.close()




if __name__ == '__main__':

    sets.img_list =   path/'train.txt'# path/'toy_data/test_ci.txt'
    sets.n_epochs = 10
    sets.no_cuda = True
    sets.gpu_id = [0]
    sets.data_root = path #/'toy_data'
    sets.pretrain_path =  path/'pretrain/resnet_50.pth'
    sets.num_workers = 0
    sets.n_seg_classes = 3
    sets.model_depth = 50
    sets.resnet_shortcut = 'B'
    sets.ci_test = False
    sets.save_intervals=50
    sets.n_epochs=15
    #sets.resume_path= 'sets.resume_path=/content/drive/My Drive/Deep Learning Final Project/trails/models/resnet_50_epoch_9_batch_299.pth.tar'
    sets.input_D = 14
    sets.input_H = 28
    sets.input_W = 28
    convert = False
    if convert:
      covert_h5(path,sets.img_list)
    
    if sys.argv[1] == 'h5':
        covert_h5(sys.argv[2])
    elif sys.argv[1] == 'label':
        label_stats(sys.argv[2])

#the new dataloader

In [ ]:
class BratsSet(Dataset):
    """ Annual Brats challenges dataset """
    # binarize: whether to binarize mask (do whole-tumor segmentation)
    # modality: if the image has multiple modalities, 
    # choose which modality to output (-1 to output all).
    # If mode == 'train' and train_loc_prob > 0, then min_output_size is necessary.
  
    def __init__(self,sets, base_dir, img_list, sample_num=None, 
                 ds_weight=1.,
                 xyz_permute=None, transform=None, 
                 chosen_modality=-1, binarize=False,
                 train_loc_prob=0, min_output_size=None):
        #super(BratsSet, self).__init__()
        super(BratsSet, self).__init__()
        self._base_dir = base_dir
        self.split = sets.phase
        self.mode = sets.phase
        self.xyz_permute = xyz_permute
        self.ds_weight = ds_weight
        
        self.transform = transform
        self.chosen_modality = chosen_modality
        self.binarize    = binarize
        self.train_loc_prob = train_loc_prob
        self.min_output_size = min_output_size
        
        if self.split == 'train':
          trainlist_filepath = sets.img_list
          with open(trainlist_filepath, 'r') as f:
            self.train_image_list = f.readlines()
        elif self.split == 'test':
          testlist_filepath  = sets.img_list
          with open(testlist_filepath,  'r') as f:
            self.test_image_list = f.readlines()

        

        if self.split == 'train':
            self.image_list = self.train_image_list
        elif self.split == 'test':
            self.image_list = self.test_image_list
        elif self.split == 'all':
            self.image_list = self.all_image_list

        self.image_list = [ item.replace('\n', '') for item in self.image_list ]
        print('self.image_list ={}'.format(self.image_list))
        if sample_num is not None:
            self.image_list = self.image_list[:sample_num]
                
        self.num_modalities = 0
        # Fetch image 0 to get num_modalities.
        sample0 = self.__getitem__(0, do_transform=False)
        if len(sample0['image'].shape) == 4:
            self.num_modalities = sample0['image'].shape[0]
        
        print("'{}' {} samples, num_modalities: {}, chosen: {}".format(self.split, 
                len(self.image_list), self.num_modalities, self.chosen_modality))

    def __len__(self):
        return len(self.image_list)
        
    def __getitem__(self, idx, do_transform=True):
        image_name = self.image_list[idx]
        
        image_path = str(self._base_dir)+ '/'+str(image_name)+ '/'+image_name[3:] + ".h5"
        
        try:
           h5f = h5py.File(image_path, 'r')
        except:
          print('the pat {} is bad'.format(image_path))
          return
        image = h5f['image'][:]
      
        mask  = h5f['label'][:]

      
        if self.num_modalities > 0 and self.chosen_modality != -1:
            image = image[self.chosen_modality, :, :, :]
        if self.binarize:
            print('binarize')
            mask = (mask >= 1).astype(np.uint8)
        else:
            # Map 4 to 3, and keep 0,1,2 unchanged.
            mask -= (mask == 4)

        if self.mode == 'train' and self.train_loc_prob > 0 \
          and np.random.random() < self.train_loc_prob:
            print('localize')
            image, mask = localize(image, mask, self.min_output_size)

        # xyz_permute by default is None.
        if do_transform and self.xyz_permute is not None:
            print('do_transform')
            image = image.transpose(self.xyz_permute)
            mask  = mask.transpose(self.xyz_permute)

        if self.mode == 'train':
          print('randome and randome noise')
          k = np.random.randint(0, 4)

          # Image has multiple modalities. First dimension is the modality.
          # Only rotate along the x,y (0,1) plane.
          if image.ndim == mask.ndim + 1:
              image = np.rot90(image, k, axes=(1,2))
          else:
              image = np.rot90(image, k, axes=(0,1))

          mask = np.rot90(mask, k, axes=(0,1))

          # Flip along a random dimension. It doesn't change the shape of a tensor.
          
          axis = np.random.randint(0, 3)
          if image.ndim == mask.ndim + 1:
              image = np.flip(image, axis=axis+1).copy()
          else:
              image = np.flip(image, axis=axis).copy()

          mask = np.flip(mask, axis=axis).copy()
          ####### randome noise##############
         
        sample = { 'image': image, 'mask': mask }
        
        
        
       
        if do_transform and self.transform:
            sample = self.transform(sample)
        
        sample['image_path'] = image_name
        sample['weight'] = self.ds_weight
       
        return sample

    def create_file_list(self, train_test_split):
        img_dirs = [ d for d in listdir(self._base_dir) if isdir(join(self._base_dir, d)) ]

        # Randomize the file list, then split. Not to use the official testing set 
        # since we don't have ground truth masks for this.
        num_files = len(img_dirs)
        idxList = np.arange(num_files)  # List of file indices
        self.imgFiles = {}
        for idx in idxList:
            self.imgFiles[idx] = join(img_dirs[idx], img_dirs[idx] + ".h5")

        with open( join(self._base_dir, 'all.list'), "w" ) as allFile:
            for img_idx in idxList:
                allFile.write("%s\n" %self.imgFiles[img_idx])
        allFile.close()
                
        idxList = np.random.permutation(idxList)  # Randomize list
        train_len = int(np.floor(num_files * train_test_split)) # Number of training files
        train_indices = idxList[0:train_len]  # List of training indices
        test_indices  = idxList[train_len:]  # List of testing indices

        with open( join(self._base_dir, 'train.list'), "w" ) as trainFile:
            for img_idx in sorted(train_indices):
                trainFile.write("%s\n" %self.imgFiles[img_idx])
        trainFile.close()
        
        with open( join(self._base_dir, 'test.list'),  "w" ) as testFile:
            for img_idx in sorted(test_indices):
                testFile.write("%s\n"  %self.imgFiles[img_idx])
        testFile.close()
        
        print("%d files are split to %d training, %d test" %(num_files, train_len, len(test_indices)))
            
class CenterCrop(object):
    def __init__(self, output_size):
        self.output_size = output_size

    def __call__(self, sample):
        image, mask = sample['image'], sample['mask']
        
        # pad the sample if necessary
        if mask.shape[0] <= self.output_size[0] or mask.shape[1] <= self.output_size[1] or mask.shape[2] <= \
                self.output_size[2]:
            ph = max((self.output_size[0] - mask.shape[0]) // 2 + 3, 0)
            pw = max((self.output_size[1] - mask.shape[1]) // 2 + 3, 0)
            pd = max((self.output_size[2] - mask.shape[2]) // 2 + 3, 0)
            mask_padding = [(ph, ph), (pw, pw), (pd, pd)]
            print('padddd')
            image_padding = mask_padding
            if len(image.shape) == 4:
                image_padding = [(0, 0)] + image_padding

            image = np.pad(image, image_padding, mode='constant', constant_values=0)
            mask  = np.pad(mask,  mask_padding,  mode='constant', constant_values=0)

        (h, w, d) = image.shape[:3]

        h1 = int(round((h - self.output_size[0]) / 2.))
        w1 = int(round((w - self.output_size[1]) / 2.))
        d1 = int(round((d - self.output_size[2]) / 2.))

        mask  = mask[h1:h1  + self.output_size[0], w1:w1 + self.output_size[1], d1:d1 + self.output_size[2]]
        if len(image.shape) == 4:
            image = image[:, h1:h1 + self.output_size[0], w1:w1 + self.output_size[1], d1:d1 + self.output_size[2]]
        else:
            image = image[h1:h1 + self.output_size[0], w1:w1 + self.output_size[1], d1:d1 + self.output_size[2]]
                
        return {'image': image, 'mask': mask}

class RandomCrop(object):
    """
    Crop randomly the image in a sample
    Args:
    output_size (int): Desired output size
    """

    def __init__(self, output_size):
        self.output_size = output_size

    def __call__(self, sample):
        image, mask = sample['image'], sample['mask']
        orig_mask_shape = list(mask.shape)
        
        # pad the sample if necessary
        if mask.shape[0] <= self.output_size[0] or mask.shape[1] <= self.output_size[1] or mask.shape[2] <= \
                self.output_size[2]:
            ph = max((self.output_size[0] - mask.shape[0]) // 2 + 3, 0)
            pw = max((self.output_size[1] - mask.shape[1]) // 2 + 3, 0)
            pd = max((self.output_size[2] - mask.shape[2]) // 2 + 3, 0)
            mask_padding = [(ph, ph), (pw, pw), (pd, pd)]
            image_padding = mask_padding
            # the modality dimension.
            if len(image.shape) == 4:
                image_padding = [(0, 0)] + image_padding
            
            try:
                image = np.pad(image, image_padding, mode='constant', constant_values=0)
                mask  = np.pad(mask,  mask_padding,  mode='constant', constant_values=0)
            except:
                pdb.set_trace()
        else:
            image_padding = 0
            
        
        (h, w, d) = image.shape[-3:]
        # if np.random.uniform() > 0.33:
        #     w1 = np.random.randint((w - self.output_size[0])//4, 3*(w - self.output_size[0])//4)
        #     h1 = np.random.randint((h - self.output_size[1])//4, 3*(h - self.output_size[1])//4)
        # else:
        h1 = np.random.randint(0, h - self.output_size[0])
        w1 = np.random.randint(0, w - self.output_size[1])
        d1 = np.random.randint(0, d - self.output_size[2])
        # print(h1, h1 + self.output_size[0], w1, w1 + self.output_size[1], d1, d1 + self.output_size[2])
            
        mask  = mask[h1:h1 + self.output_size[0], w1:w1 + self.output_size[1], d1:d1 + self.output_size[2]]
        # image: (118, 118, 102). mask: (118, 118, 102) => (112, 112, 96)
        if image.ndim == 4:
            # For a 4D image, the first dimension is modality.
            image = image[:, h1:h1 + self.output_size[0], w1:w1 + self.output_size[1], d1:d1 + self.output_size[2]]
        else:
            image = image[h1:h1 + self.output_size[0], w1:w1 + self.output_size[1], d1:d1 + self.output_size[2]]

        return {'image': image, 'mask': mask}


class RandomRotFlip(object):
    """
    Crop randomly flip the dataset in a sample
    Args:
    output_size (int): Desired output size
    """

    def __call__(self, sample):
        image, mask = sample['image'], sample['mask']
        assert (image.ndim == mask.ndim) or (image.ndim == mask.ndim + 1)
        # Rotate by k*90. k is not the axis.
        k = np.random.randint(0, 4)

        # Image has multiple modalities. First dimension is the modality.
        # Only rotate along the x,y (0,1) plane.
        if image.ndim == mask.ndim + 1:
            image = np.rot90(image, k, axes=(1,2))
        else:
            image = np.rot90(image, k, axes=(0,1))
            
        mask = np.rot90(mask, k, axes=(0,1))
        
        # Flip along a random dimension. It doesn't change the shape of a tensor.
        axis = np.random.randint(0, 3)
        if image.ndim == mask.ndim + 1:
            image = np.flip(image, axis=axis+1).copy()
        else:
            image = np.flip(image, axis=axis).copy()
            
        mask = np.flip(mask, axis=axis).copy()

        return { 'image': image, 'mask': mask }


class RandomNoise(object):
    def __init__(self, mu=0, sigma=0.1, nonzero_only=True):
        self.mu = mu
        self.sigma = sigma
        self.nonzero_only = nonzero_only
        
    def __call__(self, sample):
        image, mask = sample['image'], sample['mask']
        noise = np.clip(self.sigma * np.random.randn(*image.shape), -2*self.sigma, 2*self.sigma)
        noise = noise + self.mu
        if self.nonzero_only:
            # Only add noise to non-zero voxels
            image = image + noise * (image != 0)
        else:
            image = image + noise
            
        return {'image': image, 'mask': mask}

class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        image = sample['image']
        mask  = sample['mask']
        if image.ndim == 3:
            image = image.reshape((1,) + image.shape)
        return { 'image': torch.from_numpy(image), 
                 'mask':  torch.from_numpy(mask) }

# mask should be one-hot. shape: Batch, Class, H, W, D
def RandomResizedCrop(volume, mask, out_size, crop_percents, isotropic=True):
    H, W, D = volume.shape[-3:]
    min_crop, max_crop = crop_percents  # -0.1, 0.1
    min_scale = 1 + min_crop            # 0.9
    max_scale = 1 + max_crop            # 1.1
    
    if not isotropic:
        scale_H = torch.rand(1) * (max_scale - min_scale) + min_scale
        scale_W = torch.rand(1) * (max_scale - min_scale) + min_scale
        scale_D = torch.rand(1) * (max_scale - min_scale) + min_scale
    else:
        # scale_H is uniform between [min_scale, max_scale]
        scale_H = torch.rand(1) * (max_scale - min_scale) + min_scale
        scale_W = scale_D = scale_H

    H2 = int(H * scale_H)
    W2 = int(W * scale_W)
    D2 = int(D * scale_D)
    
    # volume: [4, 1, 112, 112, 96]. mask: [4, 4, 112, 112, 96]
    volume2 = F.interpolate(volume, size=(H2, W2, D2), mode='trilinear', align_corners=False)
    mask2   = F.interpolate(mask,   size=(H2, W2, D2), mode='trilinear', align_corners=False)
    
    # out_size: (112, 112, 96)
    H_out, W_out, D_out = out_size
    if H2 < H_out or W2 < W_out or D2 < D_out:
        pad_h  = max(H_out - H2, 0)
        pad_h1 = pad_h // 2
        pad_h2 = pad_h - pad_h1
        pad_w  = max(W_out - W2, 0)
        pad_w1 = pad_w // 2
        pad_w2 = pad_w - pad_w1
        pad_d  = max(D_out - D2, 0)
        pad_d1 = pad_d // 2
        pad_d2 = pad_d - pad_d1
        
        pads = (pad_d1, pad_d2, pad_w1, pad_w2, pad_h1, pad_h2)
        
        volume2 = F.pad(volume2, pads, "constant", 0)
        mask2   = F.pad(mask2,   pads, "constant", 0)
    
    H2, W2, D2 = volume2.shape[2:] 
    h_start = torch.randint(H2 - H_out + 1, (1,))
    h_end   = h_start + H_out
    w_start = torch.randint(W2 - W_out + 1, (1,))
    w_end   = w_start + W_out
    d_start = torch.randint(D2 - D_out + 1, (1,))
    d_end   = d_start + D_out
        
    volume3 = volume2[:, :, h_start:h_end, w_start:w_end, d_start:d_end].clone()
    mask3   = mask2[:,   :, h_start:h_end, w_start:w_end, d_start:d_end].clone()
    # If converting the continuous mask values (produced by interpolation) to one-hot, performance drops. So keep the cont
    # mask3   = (mask3 >= 0.5).type(mask.dtype)
    
    return volume3, mask3





if __name__ == '__main__':
      # settting
      sets =parse_opts() 
      sets.img_list =   path/'train.txt'# path/'toy_data/test_ci.txt'
      sets.n_epochs = 10
      sets.no_cuda = False
      sets.gpu_id = [0]
      sets.data_root = path #/'toy_data'
      sets.pretrain_path =  path/'pretrain/resnet_50.pth'
      sets.num_workers = 0
      sets.n_seg_classes = 3
      sets.model_depth = 50
      sets.resnet_shortcut = 'B'
      sets.phase = 'train'
      sets.ci_test = False
      sets.save_intervals=50
      sets.n_epochs=15
      sets.input_D = 14
      sets.input_H = 28
      sets.input_W = 28
      sets.save_folder = "/content/drive/My Drive/Deep Learning Final Project/trails/models/{}_{}".format( sets.model,  sets.model_depth )


      training_dataset = BratsSet(sets,sets.data_root, sets.img_list )

the dataloader code

trying to load it from the drive



***vizualizing the loaded data:***

In [ ]:
#@title vizualizing the loaded data
from skimage.util import montage as montage2d
from torch.utils.data import DataLoader
# getting data
data_loader = DataLoader(training_dataset, batch_size=sets.batch_size, shuffle=True, num_workers=sets.num_workers)

dataset_iter = iter(data_loader)
batch_data = next(dataset_iter)

volumes, mask = batch_data['image'], batch_data['mask']
print('vizualiztion demo: volumes.shape =  {}'.format(volumes.shape))
print('vizualiztion demo: mask.shape =  {}'.format(mask.shape))

display_img_demo = volumes[0,0] #need to cheek if its T1,flair...
display_label_demo = mask[0] #this is target for all

img_vizualization_demo_path = Path('/content/drive/My Drive/Deep Learning Final Project/MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_2_1/BraTS19_2013_2_1_t1.nii.gz')
label_vizualization_demo_path = Path('/content/drive/My Drive/Deep Learning Final Project/MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_2_1/BraTS19_2013_2_1_seg.nii.gz')

img_vizualization_demo_without_data_procesing = nibabel.load(img_vizualization_demo_path)
label_vizualization_demo_without_data_procesing = nibabel.load(label_vizualization_demo_path)

img_vizualization_demo_without_data_procesing = img_vizualization_demo_without_data_procesing.get_fdata()
label_vizualization_demo_without_data_procesing = label_vizualization_demo_without_data_procesing.get_fdata()


def vizualize_a_single_img_and_its_lable(display_img, display_label, montage_display = False):
    #exceptts img opened in nibibil that have gone throw get fdata

    print('vizualiztion: display_img.shape  =  {}'.format(display_img.shape))
    print('vizualiztion: display_label.shape  =  {}'.format(display_label.shape))

    #picking a particular slice of display img here we used display_img.shape[2]//2 (intger divsion)
    #needs to be changed depending on our implamataion of T1,flair.. or all of them combined as 4 channels
    #displaying one image (T1 or flair or..) and its totall lable of a slice:
    fig, (ax1, ax2) = plt.subplots(1,2, figsize = (12, 6))
    ax1.imshow(display_img[:, :,display_img.shape[2]//2].T,cmap ='bone') #needs to be changed depending on our implamataion of T1,flair.. or all of them combined as 4 channels
    ax1.set_title('Image')
    ax2.imshow(display_label[:, :,display_label.shape[2]//2].T,cmap ='bone')
    ax2.set_title('label')
    
    if (montage_display == True):
      #displaying the 3d display_img as a montage:
      fig, ax1 = plt.subplots(1, 1, figsize = (12, 6))
      ax1.imshow(montage2d(display_img), cmap ='bone')
      

      #displaying the 3d display_label as a montage:
      fig, ax1 = plt.subplots(1,1, figsize = (12, 6))
      ax1.imshow(montage2d(display_label), cmap ='bone')
      #fig.savefig('') #saving the label

vizualize_a_single_img_and_its_lable(display_img_demo,display_label_demo,True)
vizualize_a_single_img_and_its_lable(img_vizualization_demo_without_data_procesing,label_vizualization_demo_without_data_procesing,True)

**add the resnet models**

In [ ]:
#@title resnet models
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import math
from functools import partial

__all__ = [
    'ResNet', 'resnet10', 'resnet18', 'resnet34', 'resnet50', 'resnet101',
    'resnet152', 'resnet200'
]
# https://www.youtube.com/watch?v=DkNIBBBvcPs&list=PLhhyoLH6IjfxeoooqP9rhU3HJIAVAJ3Vz&index=19 resnet video

def conv3x3x3(in_planes, out_planes, stride=1, dilation=1):
    # 3x3x3 convolution with padding
    return nn.Conv3d(
        in_planes,
        out_planes,
        kernel_size=3,
        dilation=dilation,
        stride=stride,
        padding=dilation,
        bias=False)


def downsample_basic_block(x, planes, stride, no_cuda=False):
    out = F.avg_pool3d(x, kernel_size=1, stride=stride)
    zero_pads = torch.Tensor(
        out.size(0), planes - out.size(1), out.size(2), out.size(3),
        out.size(4)).zero_()
    if not no_cuda:
        if isinstance(out.data, torch.cuda.FloatTensor):
            zero_pads = zero_pads.cuda()

    out = Variable(torch.cat([out.data, zero_pads], dim=1))

    return out


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, dilation=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3x3(inplanes, planes, stride=stride, dilation=dilation)
        self.bn1 = nn.BatchNorm3d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3x3(planes, planes, dilation=dilation)
        self.bn2 = nn.BatchNorm3d(planes)
        self.downsample = downsample #identity_downsample in resnet video
        self.stride = stride
        self.dilation = dilation

    def forward(self, x):
        residual = x #identity in resnet video

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x) #idnetity_downsample if we need to change the shape

        out += residual
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, dilation=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv3d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm3d(planes)
        self.conv2 = nn.Conv3d(
            planes, planes, kernel_size=3, stride=stride, dilation=dilation, padding=dilation, bias=False)
        self.bn2 = nn.BatchNorm3d(planes)
        self.conv3 = nn.Conv3d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm3d(planes * 4)
        
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride
        self.dilation = dilation

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:# add identity
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(self,
                 block,    #the block
                 layers,      #list of how many times we want to use the block
                 sample_input_D,
                 sample_input_H,
                 sample_input_W,
                 num_seg_classes,
                 shortcut_type='B',
                 no_cuda = False):
        self.inplanes = 64
        self.no_cuda = no_cuda
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv3d(
            1,
            64,
            kernel_size=7,
            stride=(2, 2, 2),
            padding=(3, 3, 3),
            bias=False) #intial layers (we havint done any resnet layers yet)
            
        self.bn1 = nn.BatchNorm3d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool3d(kernel_size=(3, 3, 3), stride=2, padding=1)

        # from here we do resnet layers:
        self.layer1 = self._make_layer(block, 64, layers[0], shortcut_type) #make layer defined a few lines down / 256 at end
        self.layer2 = self._make_layer(
            block, 128, layers[1], shortcut_type, stride=2) #512 at end
        self.layer3 = self._make_layer(
            block, 256, layers[2], shortcut_type, stride=1, dilation=2) #1024
        self.layer4 = self._make_layer(
            block, 512, layers[3], shortcut_type, stride=1, dilation=4) #2048

        print('num_seg_classes= {}'.format(num_seg_classes))

        self.conv_seg = nn.Sequential( # this is the specific implamantation for our segmentation problam
                                        nn.ConvTranspose3d(
                                        512 * block.expansion,
                                        32,
                                        2,
                                        stride=2
                                        ),
                                        nn.BatchNorm3d(32),
                                        nn.ReLU(inplace=True),
                                        nn.Conv3d(
                                        32,
                                        32,
                                        kernel_size=3,
                                        stride=(1, 1, 1),
                                        padding=(1, 1, 1),
                                        bias=False), 
                                        nn.BatchNorm3d(32),
                                        nn.ReLU(inplace=True),
                                        nn.Conv3d(
                                        32,
                                        num_seg_classes,
                                        kernel_size=1,
                                        stride=(1, 1, 1),
                                        bias=False) )
                                        
        
                                      
                                       

        for m in self.modules():
            if isinstance(m, nn.Conv3d):
                m.weight = nn.init.kaiming_normal(m.weight, mode='fan_out')
            elif isinstance(m, nn.BatchNorm3d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, shortcut_type, stride=1, dilation=1):# blocks = num of times we use the block, plans=out channels.
        downsample = None #identity downsample
        if stride != 1 or self.inplanes != planes * block.expansion: # when do we do identity downsample ie when conv layer change the identity (the input size changed)
            if shortcut_type == 'A':
                downsample = partial(
                    downsample_basic_block,
                    planes=planes * block.expansion,
                    stride=stride,
                    no_cuda=self.no_cuda)
            else:
                downsample = nn.Sequential(
                    nn.Conv3d(
                        self.inplanes,
                        planes * block.expansion,
                        kernel_size=1,
                        stride=stride,
                        bias=False), nn.BatchNorm3d(planes * block.expansion))# here we implemented the identity down sample

        layers = []
        layers.append(block(self.inplanes, planes, stride=stride, dilation=dilation, downsample=downsample))#this layer changes the number of channels firs layer changes to 256 at the end of the block (here changes stride or/and num of channels)
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes, dilation=dilation)) #256->64, 64*4(256 again) so no identity down sample in this case

        return nn.Sequential(*layers) # unpack the list

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.conv_seg(x)
        

        return x

def resnet10(**kwargs):
    """Constructs a ResNet-18 model.
    """
    model = ResNet(BasicBlock, [1, 1, 1, 1], **kwargs)
    return model


def resnet18(**kwargs):
    """Constructs a ResNet-18 model.
    """
    model = ResNet(BasicBlock, [2, 2, 2, 2], **kwargs)
    return model


def resnet34(**kwargs):
    """Constructs a ResNet-34 model.
    """
    model = ResNet(BasicBlock, [3, 4, 6, 3], **kwargs)
    return model


def resnet50(**kwargs):
    """Constructs a ResNet-50 model.
    """
    model = ResNet(Bottleneck, [3, 4, 6, 3], **kwargs)
    return model


def resnet101(**kwargs):
    """Constructs a ResNet-101 model.
    """
    model = ResNet(Bottleneck, [3, 4, 23, 3], **kwargs)
    return model


def resnet152(**kwargs):
    """Constructs a ResNet-101 model.
    """
    model = ResNet(Bottleneck, [3, 8, 36, 3], **kwargs)
    return model


def resnet200(**kwargs):
    """Constructs a ResNet-101 model.
    """
    model = ResNet(Bottleneck, [3, 24, 36, 3], **kwargs)
    return model

# def test():
#   net = resnet50() try all of the resnets
#   x=torch.randn(2,image_dim needs to be cheeked channels and 3d dim)
#   y =net(x).to('cuda')
#   print (y.shape)
#test()

**add the model creating code**

In [ ]:

import torch
from torch import nn



def generate_model(opt):
    assert opt.model in [
        'resnet'
    ]

    if opt.model == 'resnet':
        assert opt.model_depth in [10, 18, 34, 50, 101, 152, 200]
        
        if opt.model_depth == 10:
            model = resnet10(
                sample_input_W=opt.input_W,
                sample_input_H=opt.input_H,
                sample_input_D=opt.input_D,
                shortcut_type=opt.resnet_shortcut,
                no_cuda=opt.no_cuda,
                num_seg_classes=opt.n_seg_classes)
        elif opt.model_depth == 18:
            model = resnet18(
                sample_input_W=opt.input_W,
                sample_input_H=opt.input_H,
                sample_input_D=opt.input_D,
                shortcut_type=opt.resnet_shortcut,
                no_cuda=opt.no_cuda,
                num_seg_classes=opt.n_seg_classes)
        elif opt.model_depth == 34:
            model = resnet34(
                sample_input_W=opt.input_W,
                sample_input_H=opt.input_H,
                sample_input_D=opt.input_D,
                shortcut_type=opt.resnet_shortcut,
                no_cuda=opt.no_cuda,
                num_seg_classes=opt.n_seg_classes)
        elif opt.model_depth == 50:
            model = resnet50(
                sample_input_W=opt.input_W,
                sample_input_H=opt.input_H,
                sample_input_D=opt.input_D,
                shortcut_type=opt.resnet_shortcut,
                no_cuda=opt.no_cuda,
                num_seg_classes=opt.n_seg_classes)
        elif opt.model_depth == 101:
            model = resnet.resnet101(
                sample_input_W=opt.input_W,
                sample_input_H=opt.input_H,
                sample_input_D=opt.input_D,
                shortcut_type=opt.resnet_shortcut,
                no_cuda=opt.no_cuda,
                num_seg_classes=opt.n_seg_classes)
        elif opt.model_depth == 152:
            model = resnet.resnet152(
                sample_input_W=opt.input_W,
                sample_input_H=opt.input_H,
                sample_input_D=opt.input_D,
                shortcut_type=opt.resnet_shortcut,
                no_cuda=opt.no_cuda,
                num_seg_classes=opt.n_seg_classes)
        elif opt.model_depth == 200:
            model = resnet.resnet200(
                sample_input_W=opt.input_W,
                sample_input_H=opt.input_H,
                sample_input_D=opt.input_D,
                shortcut_type=opt.resnet_shortcut,
                no_cuda=opt.no_cuda,
                num_seg_classes=opt.n_seg_classes)
    
    if not opt.no_cuda:
        if len(opt.gpu_id) > 1:
            model = model.cuda() 
            model = nn.DataParallel(model, device_ids=opt.gpu_id)
            net_dict = model.state_dict() 
        else:
            import os
            
            print('running with cuda')
            os.environ["CUDA_VISIBLE_DEVICES"]=str(opt.gpu_id[0])
            model = model.cuda() 
            model = nn.DataParallel(model, device_ids=None)
            net_dict = model.state_dict()
    else:
       
        net_dict = model.state_dict()
    
    # load pretrain
    #if opt.phase != 'test' and opt.pretrain_path:
    if  opt.pretrain_path:
        print ('loading pretrained model {}'.format(opt.pretrain_path))
        print('test otzi')
        if not opt.no_cuda:
          pretrain = torch.load(opt.pretrain_path)
        else:
          pretrain = torch.load(opt.pretrain_path, map_location='cpu')
        pretrain_dict = {k: v for k, v in pretrain['state_dict'].items() if k in net_dict.keys()}
         
        net_dict.update(pretrain_dict)
        model.load_state_dict(net_dict)

        new_parameters = [] 
        for pname, p in model.named_parameters():
            for layer_name in opt.new_layer_names:
                if pname.find(layer_name) >= 0:
                    new_parameters.append(p)
                    break
        
        new_parameters_id = list(map(id, new_parameters))
        base_parameters = list(filter(lambda p: id(p) not in new_parameters_id, model.parameters()))
        parameters = {'base_parameters': base_parameters, 
                      'new_parameters': new_parameters}
        '''
        for param in parameters:
           param.requires_grad = False  ### moved here by avi insteed of in training methode
        '''
        print('returned parameters')
        return model, parameters

    print('returned model.parameter')
    return model, model.parameters()

adding a utils func

In [ ]:

class SmoothDiceLoss(nn.Module):
    # Set momentum = 1 to disable smoothing.
    def __init__(self, momentum=0.1):
        super(SmoothDiceLoss, self).__init__()
        self.momentum = momentum
        self.running_denom = -1
        self.eps = 1e-5
        
    def forward(self, score, gt_mask):
        score = score.view(score.shape[0], -1)
        gt_mask = gt_mask.float().view(gt_mask.shape[0], -1)        
        intersect = torch.sum(score * gt_mask, dim=1)
        y_sum = torch.sum(gt_mask * gt_mask, dim=1)
        z_sum = torch.sum(score * score, dim=1)
        denom = z_sum + y_sum + self.eps
        mean_denom = denom.mean()
        
        if self.running_denom == -1:
            self.running_denom = mean_denom.item()
            dyn_offset = torch.zeros(1, device='cuda')
        else:
            # Update the running average of the denominator.
            self.running_denom = self.running_denom * (1 - self.momentum) \
                                 + mean_denom.item() * self.momentum
            # dyn_offset is a tensor of length nBatch.
            dyn_offset = self.running_denom - denom.data
        # Make denom + dyn_offset = self.running_denom.
        smooth_dice = (2 * intersect + self.eps + dyn_offset) / (denom + dyn_offset)
        smooth_loss = 1 - smooth_dice
        # odice: original dice. oloss: original loss.
        orig_dice = (2 * intersect + self.eps) / denom
        orig_loss = 1 - orig_dice
        
        smooth_loss = smooth_loss.mean()
        orig_loss   = orig_loss.mean()
        #print("r-denom: %.1f, denom: %s, offset: %s" % \
        #        (self.running_denom, str(denom.data.cpu().numpy()), str(dyn_offset.data.cpu().numpy())))
        return smooth_loss, orig_loss

# Compute each example's dice loss in a batch, and average them                
def dice_loss_indiv(score, gt_mask, weight=None):
    score = score.view(score.shape[0], -1)
    gt_mask = gt_mask.float().view(gt_mask.shape[0], -1)
    smooth = 1e-5
    intersect = torch.sum(score * gt_mask, dim=1)
    y_sum = torch.sum(gt_mask * gt_mask, dim=1)
    z_sum = torch.sum(score * score, dim=1)
    dice = (2 * intersect + smooth) / (z_sum + y_sum + smooth)
    loss = 1 - dice
    if weight is not None:
        loss = (loss * weight).mean()
    else:
        loss = loss.mean()
    return loss

# Treat the whole batch as one big example, and compute the dice loss.
def dice_loss_mix(score, gt_mask):
    gt_mask = gt_mask.float()
    smooth = 1e-5
    intersect = torch.sum(score * gt_mask)
    y_sum = torch.sum(gt_mask)
    z_sum = torch.sum(score)
    loss = (2 * intersect + smooth) / (z_sum + y_sum + smooth)
    loss = 1 - loss
    return loss

# vcdr: vertical cup/disc ratio (non-differentiable).
# mask_nhot_soft: [B, C, H, W]. 
# mask_nhot_soft can also be a hard (groundtruth) mask, as threadholding doesn't change it.

                    


**OuerNewModel:** **bold text**

In [ ]:
#@title OuerNewModel
class DoubleConvdown(nn.Module):
    """(convolution => [IN] => leakyReLU) * 2"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.down_scale=nn.Sequential(nn.Conv3d(in_channels,out_channels,kernel_size=3, stride=1),
			    nn.InstanceNorm3d(3),
			    nn.LeakyReLU(),
        	nn.Conv3d(out_channels,out_channels, kernel_size=3, stride=1, padding=1, bias=False),
			    nn.InstanceNorm3d(3),
          nn.LeakyReLU())
        
    def forward(self, x):
        return self.down_scale(x)
class OuerNewModel(nn.Module):
    
    def __init__(self, my_pretrained_model,sets):
      
        
        super(OuerNewModel, self).__init__()
        self.my_new_layers = nn.Sequential(nn.Conv3d(4, 3, 3, stride=2),
                                           nn.ReLU(),nn.Conv3d(3, 2, 3, stride=2),
                                           nn.Conv3d(2, 1, 3, stride=1),
                                           nn.ReLU())
        self.pretrained = my_pretrained_model
        

        
        self.pool3d = nn.MaxPool3d(kernel_size=2, stride=2)
        
        # up the sys from ConvTranspose3d to half the channels and do 2 conv3d one from in_ch to out_ch and one out_ch to out_ch
        self.up_scale1=nn.Sequential(
			    nn.ConvTranspose3d(4*sets.n_seg_classes,2*sets.n_seg_classes,kernel_size=2, stride=2),
			    # should be feat_in*2 or feat_in
			    nn.Conv3d(2*sets.n_seg_classes, sets.n_seg_classes, kernel_size=3, stride=1, padding=1, bias=False),
			    nn.InstanceNorm3d(3),
			    nn.LeakyReLU(),
        	nn.Conv3d(sets.n_seg_classes, sets.n_seg_classes, kernel_size=3, stride=1, padding=1, bias=False),
			    nn.InstanceNorm3d(3),
          nn.LeakyReLU())
        #self.up_scale2=norm_lrelu_upscale_conv_norm_lrelu()
        
        self.up_scale2=nn.Sequential(
			    nn.ConvTranspose3d(sets.n_seg_classes,sets.n_seg_classes,kernel_size=3, stride=2),
			    # should be feat_in*2 or feat_in
			    nn.Conv3d(sets.n_seg_classes, sets.n_seg_classes, kernel_size=3, stride=1, padding=1, bias=False),
			    nn.InstanceNorm3d(sets.n_seg_classes),
			    nn.LeakyReLU(),
          nn.Conv3d(sets.n_seg_classes, sets.n_seg_classes, kernel_size=3, stride=1, padding=1, bias=False),
			    nn.InstanceNorm3d(3),
          nn.LeakyReLU())
      
        self.after_cat = nn.Sequential( nn.ConvTranspose3d(sets.n_seg_classes*2,sets.n_seg_classes,kernel_size=1, stride=1),  nn.Conv3d(sets.n_seg_classes, sets.n_seg_classes, kernel_size=3, stride=1, padding=1, bias=False),
			    nn.InstanceNorm3d(sets.n_seg_classes),
			    nn.LeakyReLU(),
          nn.Conv3d(sets.n_seg_classes, sets.n_seg_classes, kernel_size=3, stride=1, padding=1, bias=False),
			    nn.InstanceNorm3d(3),
          nn.LeakyReLU(),
          nn.Conv3d(sets.n_seg_classes, sets.n_seg_classes, kernel_size=3, stride=1, padding=1, bias=False),
			    nn.InstanceNorm3d(3),
          nn.LeakyReLU())
        
        self.up_scale3=nn.Sequential(
			    nn.ConvTranspose3d(8*sets.n_seg_classes,4*sets.n_seg_classes,kernel_size=2, stride=2),
			    # should be feat_in*2 or feat_in
			    nn.Conv3d(4*sets.n_seg_classes, 2*sets.n_seg_classes, kernel_size=3, stride=1, padding=1, bias=False),
			    nn.InstanceNorm3d(3),
			    nn.LeakyReLU(),
        	nn.Conv3d(2*sets.n_seg_classes, 2*sets.n_seg_classes, kernel_size=3, stride=1, padding=1, bias=False),
			    nn.InstanceNorm3d(3),
          nn.LeakyReLU())
          
        
       
        self.MaxPool3d =  nn.MaxPool3d(kernel_size=2, stride=2)
        self.down1 =  DoubleConvdown(sets.n_seg_classes, 2*sets.n_seg_classes)
        self.down2 =  DoubleConvdown(2*sets.n_seg_classes, 4*sets.n_seg_classes)
        self.down3 = DoubleConvdown(4*sets.n_seg_classes, 8*sets.n_seg_classes)
   
    def forward(self, x):
        print('x.shape before entry to my new layers:{} '.format(x.shape))
        print('x1_img1.shape before entry to my new layers:{} '.format(x[:,0,:,:,:].unsqueeze(1).shape))
        x1_img1 = self.pretrained(x[:,0,:,:,:].unsqueeze(1))
        print('x1_img1.shape in fowerd pass of till entry to pretrained resnet:{} '.format(x1_img1.shape))
        x1_img2 = self.pretrained(x[:,1,:,:,:].unsqueeze(1))
        print('x1_img2.shape in fowerd pass of till entry to pretrained resnet:{} '.format(x1_img2.shape))
        x1_img3 = self.pretrained(x[:,2,:,:,:].unsqueeze(1))
        print('x1_img3.shape in fowerd pass of till entry to pretrained resnet:{} '.format(x1_img3.shape))
        x1_img4 = self.pretrained(x[:,3,:,:,:].unsqueeze(1))
        print('x1_img4.shape in fowerd pass of till entry to pretrained resnet:{} '.format(x1_img4.shape))
        
        x2 =torch.cat((x1_img1, x1_img2), dim=1)
        x2 = torch.cat((x2, x1_img3), dim=1)
        x2 = torch.cat((x2, x1_img4), dim=1)
        print('x2.shape in fowerd pass after passing the 4 images throw the pretrained and concatnted: {}'.format(x2.shape))
        x2_conv = self.down3(x2)
        
        print('x2_conv.shape in fowerd pass after passing the 4 images throw the pretrained and concatnted: {}'.format(x2_conv.shape))


        x_downsized1 =  self.down1(x)
        x_downsized1 = self.MaxPool3d(x_downsized1)
        print('x_downsized1.shape : {}'.format(x_downsized1.shape))

        x_downsized2 =  self.down2(x_downsized1)
        x_downsized2 = self.MaxPool3d(x_downsized2)
       
        print('x_downsized2.shape : {}'.format(x_downsized2.shape))
        
        x_downsized3 =  self.down3(x_downsized2)
        
        print('x_downsized3.shape : {}'.format(x_downsized3.shape))
        
        #concatinting x_downsized2 to x2:
        x_downsized2_reshaped = torch.nn.functional.interpolate(x_downsized2,x2.shape[2:], scale_factor=None, mode='nearest')
        print('x_downsized2_reshaped.shape : {}'.format(x_downsized2_reshaped.shape))
        x_downsized2_concatinted = torch.cat((x_downsized2_reshaped,x2),dim=1)
        print('x_downsized2_concatinted.shape : {}'.format(x_downsized2_concatinted.shape))
        x_downsized2_concatinted_up3 = self.up_scale3(x_downsized2_concatinted)
        print('x_downsized2_concatinted_up3.shape : {}'.format(x_downsized2_concatinted_up3.shape))

        #concatinting x_downsized1 to x_downsized2_concatinted_up3:
        x_downsized1_reshaped = torch.nn.functional.interpolate(x_downsized1,x_downsized2_concatinted_up3.shape[2:], scale_factor=None, mode='nearest')
        print('x_downsized1_reshaped.shape : {}'.format(x_downsized1_reshaped.shape))
        x_downsized1_concatinted = torch.cat((x_downsized1_reshaped,x_downsized2_concatinted_up3),dim=1)
        print('x_downsized1_concatinted.shape : {}'.format(x_downsized1_concatinted.shape))
        x_downsized1_concatinted_up1 = self.up_scale1(x_downsized1_concatinted)
        print('x_downsized1_concatinted_up1.shape : {}'.format(x_downsized1_concatinted_up1.shape))


        #concatinting x to x_downsized1_concatinted_up1:
        x_downsized1_concatinted_up1_reshaped = torch.nn.functional.interpolate(x_downsized1_concatinted_up1,x.shape[2:], scale_factor=None, mode='nearest')
        print('x_downsized1_concatinted_up1_reshaped.shape : {}'.format(x_downsized1_concatinted_up1_reshaped.shape))
        x_reshaped_concatinted = torch.cat((x,x_downsized1_concatinted_up1_reshaped),dim=1)
        print('x_reshaped_concatinted.shape : {}'.format(x_reshaped_concatinted.shape))
        x_reshaped_concatinted_up1 = self.after_cat(x_reshaped_concatinted)
        print('x_reshaped_concatinted_up1 ,aftercat.shape after the last conv  : {}'.format(x_reshaped_concatinted_up1.shape))
        #x_out_reshped_to_x =torch.nn.functional.interpolate(x_reshaped_concatinted_up1,x.shape[2:], scale_factor=None, mode='nearest')
        #print('x_out_reshped_to_x .shape  : {}'.format(x_out_reshped_to_x.shape))


       
        return x_reshaped_concatinted_up1


    
    
def norm_lrelu_upscale_conv_norm_lrelu(self, size):
        
		    return nn.Sequential(
			    nn.InstanceNorm3d(feat_in),
			    nn.LeakyReLU(),
			    nn.Upsample(size, mode='nearest'),
			    # should be feat_in*2 or feat_in
			    nn.Conv3d(feat_in, feat_out, kernel_size=3, stride=1, padding=1, bias=False),
			    nn.InstanceNorm3d(feat_out),
			    nn.LeakyReLU())

  

**test the model**

In [ ]:
#model, parameters = generate_model(sets) 

**add Utils**

In [ ]:
#@title load_lines



import logging

logging.basicConfig(
    format='%(asctime)s %(levelname)-8s [%(filename)s:%(lineno)d] %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    level=logging.DEBUG)

log = logging.getLogger()

import os
import os.path as osp

def load_lines(file_path):
    """Read file into a list of lines.
    Input
      file_path: file path
    Output
      lines: an array of lines
    """
    with open(file_path, 'r') as fio:
        lines = fio.read().splitlines()
    return lines

**check the train code**

In [ ]:
#@title  train code
from torch import nn
import itertools
from torch import optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
import time

from scipy import ndimage
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"


def train(data_loader, model, optimizer, scheduler, total_epochs, save_interval, save_folder, sets):
    # settings
    batches_per_epoch = len(data_loader)
    log.info('{} epochs in total, {} batches per epoch'.format(total_epochs, batches_per_epoch))
    
    loss_seg = nn.CrossEntropyLoss()
    bce_loss_func = nn.BCEWithLogitsLoss( pos_weight=sets.bce_weight.cuda())
    
    counter=0
    best_loss=1
    losses=[]

   

    print("Current setting is:")
    print(sets)
    print("\n\n")     
    if not sets.no_cuda:
        loss_seg = loss_seg.cuda()
        
    model.train()
    train_time_sp = time.time()

    class_weights = torch.ones(sets.n_seg_classes).cuda()
    class_weights[0] = 0
    
    class_weights /= class_weights.sum()
    
    for epoch in range(total_epochs):
        log.info('Start epoch {}'.format(epoch))
        
        scheduler.step()

        #------moved the schduler to be after the optimizer------


        log.info('lr = {}'.format(scheduler.get_lr()))
        print('len data_loader = {} '.format(len(data_loader)))
        for batch_id, batch_data in enumerate(data_loader):
            dice_losses = []
            total_dice_loss = 0
            
            
            
            # getting data batch
            counter+=1
            batch_id_sp = epoch * batches_per_epoch
           
            if not sets.no_cuda: 
                volumes, mask = batch_data['image'].cuda(), batch_data['mask'].cuda()
                print('cuda')
            else:
              volumes, mask = batch_data['image'], batch_data['mask']

            print('volumes.shape ={}'.format(volumes.shape ))
            print('mask.shape ={}'.format(mask.shape ))
            mask = brats_map_label(mask, sets.n_seg_classes)
            print('mask.shape after brats map label={}'.format(mask.shape ))
            mask = torch.tensor(mask)
            mask = mask.permute(1, 0, 2,3,4)
            print('mask.shape after premute ={}'.format(mask.shape ))
            

          
           

            optimizer.zero_grad()
            out_masks = model(volumes)
            print('out_masks.shape ={}'.format(out_masks.shape ))
            

            !CUDA_LAUNCH_BLOCKING=1
            label_masks= mask

            DICE_W =0.2

           
            outputs_soft    = torch.sigmoid(out_masks)
            
            print('outputs_soft.shape ={}'.format(outputs_soft.shape ))
            
            
             

           

            print('out_masks.shape before ce loss ={}'.format(out_masks.shape ))
            print('mask.shape before ce loss={}'.format(mask.shape ))
            print('out_masks.shape before ce loss ={}'.format(out_masks.shape ))
            print('mask.shape before ce loss={}'.format(mask.shape ))
            #----- trying omthing new ----
            total_ce_loss   = bce_loss_func(out_masks.permute([0,2,3,4,1]), 
                                            # after brats_map_label(), dim 1 of mask_batch is segmantation class.
                                            # It's permuted to the last dim to align with outputs for bce loss computation.
                                            mask.permute([0,2,3,4,1]))


            
            

            
            ### calculaiting the dice loss for evry chanel
            for cls in range(1, sets.n_seg_classes):
                allcls_pred_np  = outputs_soft.data.cpu().numpy()
                allcls_gt_np    = mask.data.cpu().numpy()
                pred = allcls_pred_np[:, cls].astype(np.uint8)
                gt   = allcls_gt_np[ :, cls].astype(np.uint8)
                dice_loss = 1- metric.binary.dc(gt, pred)
                dice_losses.append(dice_loss)
                total_dice_loss = total_dice_loss + dice_loss * class_weights[cls]
            
            # calculating loss
            
            loss = (1 - DICE_W) * total_ce_loss + DICE_W * total_dice_loss
            print('total_ce_loss={}'.format(total_ce_loss))
            print('total_dice_loss={}'.format(total_dice_loss))
            losses.append(loss)
            

            optimizer.zero_grad()
            loss.backward()
            
            
            optimizer.step()

            avg_batch_time = (time.time() - train_time_sp) / (1 + batch_id_sp)
            log.info(
                    'Batch: {}-{} ({}), loss = {:.3f}, loss_seg = {:.3f}, avg_batch_time = {:.3f}'\
                    .format(epoch, batch_id, batch_id_sp, loss.item(), loss.item(), avg_batch_time))
          
            if not sets.ci_test:
                # save model
                if  counter != 0 and counter % save_interval == 0 :
                
                    model_save_path = '{}_epoch_{}_batch_{}.pth.tar'.format(save_folder, epoch, batch_id)
                    best_loss = loss
                    model_save_dir = os.path.dirname(model_save_path)
                    if not os.path.exists(model_save_dir):
                        os.makedirs(model_save_dir)
                    
                    log.info('Save checkpoints: epoch = {}, batch_id = {}'.format(epoch, batch_id)) 
                    print('saved the model in {}'.format(model_save_path))
                    sets.resume_path= model_save_path
                    torch.save({
                                'ecpoch': epoch,
                                'batch_id': batch_id,
                                'state_dict': model.state_dict(),
                                'optimizer': optimizer.state_dict()},
                                model_save_path)
                    #print('saved the model')
                            
    print('Finished training')
    xaxis= list(range(0, counter))
    plt.plot(xaxis,losses)
    plt.title(' Loss ')

    # naming the x axis
    plt.xlabel('Pa')
    # naming the y axis
    plt.ylabel('Loss')

    # function to show the plot
    plt.show()            
    if sets.ci_test:
        exit()


if __name__ == '__main__':
      
      sets =parse_opts()  

      print('passed sets')


      sets.img_list = path/'train.txt'
      sets.no_cuda = False
      sets.gpu_id = [0]
      sets.data_root = path 
      sets.pretrain_path =  path/'pretrain/resnet_10.pth'
      sets.num_workers = 0
      sets.n_seg_classes = 4
      sets.model_depth = 10
      sets.resnet_shortcut = 'B'
      sets.ci_test = False
      sets.save_intervals=10
      sets.n_epochs=10
      sets.batch_size = 1
     
      sets.input_D = 120 #14
      sets.input_H = 130 #28
      sets.input_W = 130 #28
      sets.orig_patch_size= '152,160,152'

      #-- added somthing for the test, this is taken from https://github.com/askerlee/segtran/blob/master/code/train3d.py train code
      sets.bce_weight =[0., 3, 1, 1.75]  # bg, ET, WT, TC

      #sets.bce_weight = torch.tensor(sets.bce_weight).cuda()
      sets.bce_weight = torch.tensor(sets.bce_weight).cpu()
      sets.bce_weight = sets.bce_weight * (sets.n_seg_classes - 1) / sets.bce_weight.sum()


      
      sets.save_folder = "/content/drive/My Drive/Deep Learning Final Project/trails/models/{}_{}".format( sets.model,  sets.model_depth )
    
      
 
     
    
      torch.manual_seed(sets.manual_seed)
      class Identity(nn.Module):
          def __init__(self):
            super(Identity, self).__init__()

          def forward(self, x):
              return x
      
      model, parameters = generate_model(sets)
      model.conv_seg = Identity()
      
      #for param in model.parameters():
      #    param.requires_grad = False
      #for name, param in model.named_parameters():
      #    print('name = {}'.format(name))
      #    param.requires_grad = False
      #    if name == 'module.conv_seg.6.weight':
      #      param.requires_grad = True
      #      print('{} with  grad'.format(name))

      
      
      model=OuerNewModel(model,sets)
      #print (model)
      if torch.cuda.is_available():
        model.cuda()
      
      # optimizer
      if sets.ci_test:
          params = [{'params': parameters, 'lr': sets.learning_rate}]
      else:
          params = [
                  { 'params': parameters['base_parameters'], 'lr': sets.learning_rate }, 
                  { 'params': parameters['new_parameters'], 'lr': sets.learning_rate*100 }
                  ]
      
      optimizer = torch.optim.Adam(model.parameters(),lr=sets.learning_rate)#learning_rate)
          
      scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.99)
      

      # train from resume
      if sets.resume_path:
          
          if os.path.isfile(sets.resume_path):
              print('resumed the path')
              print("=> loading checkpoint '{}'".format(sets.resume_path))
              checkpoint = torch.load(sets.resume_path)
              model.load_state_dict(checkpoint['state_dict'])
              optimizer.load_state_dict(checkpoint['optimizer'])
              print("=> loaded checkpoint '{}' (epoch {})"
                .format(sets.resume_path, checkpoint['epoch']))
              print('loaded the model')
              

      # getting data
      sets.phase = 'train'
      if sets.no_cuda:
          sets.pin_memory = False
      else:
          sets.pin_memory = True    
      
      training_dataset = BratsSet(sets,sets.data_root, sets.img_list)
      data_loader = DataLoader(training_dataset, batch_size=sets.batch_size, shuffle=True, num_workers=sets.num_workers, pin_memory=sets.pin_memory)
      

      
##############################################
      print(sets.save_folder)
      # training
      train(data_loader, model, optimizer, scheduler, total_epochs=sets.n_epochs, save_interval=sets.save_intervals, save_folder=sets.save_folder, sets=sets) 

In [ ]:
vizualize the network

**add the tset func**

In [ ]:
      dataset_iter = iter(data_loader)
      batch_data = next(dataset_iter)
      #print('data_loader = {}'.format(data_loader))
      if not sets.no_cuda: 
          volumes, mask = batch_data['image'].cuda(), batch_data['mask'].cuda()
          print('cuda')
      else:
        volumes, mask = batch_data['image'], batch_data['mask']
      
      print('volumes.shape before fowered pass ={}'.format(volumes.shape ))
      print('mask.shape before fowered pass ={}'.format(mask.shape ))
      mask = brats_map_label(mask, sets.n_seg_classes)
      print('mask.shape after brats map label={}'.format(mask.shape ))
      mask = torch.tensor(mask)
      mask = mask.permute(1, 0, 2,3,4)
      print('mask.shape after premute ={}'.format(mask.shape ))
      optimizer.zero_grad()
      out_masks = model(volumes)
      print('out_masks.shape ={}'.format(out_masks.shape ))

      outputs_soft    = torch.sigmoid(out_masks)
     
      outputs_soft = torch.where(outputs_soft>0.5, 1, 0) 
      out_masks = outputs_soft
      print('outputs_soft.shape ={}'.format(outputs_soft.shape ))
      print('vizualize volumes and label AFTER fowerd pass channel 0:')
      vizualize_a_single_img_and_its_lable(out_masks[0,0].cpu().detach().numpy(), mask[0,0].cpu(), montage_display = True)
      print('vizualize volumes and label AFTER fowerd pass channel 1:')
      vizualize_a_single_img_and_its_lable(out_masks[0,1].cpu().detach().numpy(), mask[0,1].cpu(), montage_display = True)
      print('vizualize volumes and label AFTER fowerd pass channel 2:')
      vizualize_a_single_img_and_its_lable(out_masks[0,2].cpu().detach().numpy(), mask[0,2].cpu(), montage_display = True)
      print('vizualize volumes and label AFTER fowerd pass channel 3:')
      vizualize_a_single_img_and_its_lable(out_masks[0,3].cpu().detach().numpy(), mask[0,3].cpu(), montage_display = True)
   
   
   
         

In [ ]:
#@title  test code

def test_calculate_metric(sets,iter_nums,net,db_test):
   
   
    loss1_WT=0
    loss1_ET=0
    loss1_TC=0
    loss2_WT=0
    loss2_ET=0
    loss2_TC=0
 
    counter=0
    net.cuda()
    net.eval()
    preproc_fn = None
    test_save_paths=[None]
    has_mask =    False

    for batch_id, batch_data in enumerate(data_loader):

    
        
      if not sets.no_cuda: 
                volumes, mask = batch_data['image'].cuda(), batch_data['mask'].cuda()
                print('cuda')
      else:
              volumes, mask = batch_data['image'], batch_data['mask']
      counter+=1 
      print('volumes.shape ={}'.format(volumes.shape ))
      print('mask.shape ={}'.format(mask.shape ))
      mask = brats_map_label(mask, sets.n_seg_classes)
      print('mask.shape after brats map label={}'.format(mask.shape ))
      mask = torch.tensor(mask)
      mask = mask.permute(1, 0, 2,3,4)
      print('mask.shape after premute ={}'.format(mask.shape )) 
      out_masks = model(volumes)
      print('out_masks.shape ={}'.format(out_masks.shape ))
      outputs_soft    = torch.sigmoid(out_masks)
      outputs_soft = torch.where(outputs_soft>0.5, 1, 0) 
      print('outputs_soft.shape ={}'.format(outputs_soft.shape ))
      dice_losses1 = []
      dice_losses2 = []
      total_dice_loss1 = 0
      total_dice_loss2 = 0
      for cls in range(1, sets.n_seg_classes):
                if counter!=0 and counter% sets.save_intervals ==0:
                  print('vizualize volumes and label AFTER fowerd pass channel {}:'.format(cls))
                  vizualize_a_single_img_and_its_lable(outputs_soft[0, cls].cpu().detach().numpy(), mask[0, cls].cpu().detach().numpy(), montage_display = True)
                  
                 
      loss1_WT= (loss1_WT*(counter-1)+dice_loss_indiv(outputs_soft[:, 1], mask[ :, 1]))/counter
      loss1_ET=(loss1_ET*(counter-1)+dice_loss_indiv(outputs_soft[:, 2], mask[ :, 2]))/counter
      loss1_TC=(loss1_ET*(counter-1)+dice_loss_indiv(outputs_soft[:, 3], mask[ :, 3]))/counter
      allcls_pred_np  = outputs_soft.data.cpu().numpy()
      allcls_gt_np    = mask.data.cpu().numpy()
      pred = allcls_pred_np[:, 1].astype(np.uint8)
      gt   = allcls_gt_np[ :, 1].astype(np.uint8)
      loss2_WT= (loss2_WT*(counter-1)+(1-metric.binary.dc(gt, pred)))/counter
      pred = allcls_pred_np[:, 2].astype(np.uint8)
      gt   = allcls_gt_np[ :, 2].astype(np.uint8)
      loss2_ET= (loss2_ET*(counter-1)+(1-metric.binary.dc(gt, pred)))/counter
      pred = allcls_pred_np[:, 3].astype(np.uint8)
      gt   = allcls_gt_np[ :, 3].astype(np.uint8)
      loss2_TC= (loss2_TC*(counter-1)+(1-metric.binary.dc(gt, pred)))/counter



    print('loss1_WT={}'.format(loss1_WT))
    print('loss1_TC={}'.format(loss1_TC)) 
    print('loss1_ET={}'.format(loss1_ET))
    print('loss2_WT={}'.format(loss2_WT))
    print('loss2_TC={}'.format(loss2_TC)) 
    print('loss2_ET={}'.format(loss2_ET))          
    return loss1_WT,loss1_WT,counter


if __name__ == '__main__':

    # settting
      sets = parse_opts()
      sets.target_type = "normal"
      sets.phase = 'train'
      sets.resume_path = '/content/drive/My Drive/Deep Learning Final Project/trails/models/resnet_10_epoch_3_batch_159.pth.tar'
      sets.pretrain_path =  path/'pretrain/resnet_10.pth'
      sets.img_list =   path/'valid.txt'
      sets.model_depth = 10
      sets.n_seg_classes = 4
      sets.save_intervals=10
      sets.gpu_id = [0]
      sets.orig_patch_size= [136,152,136]
      sets.no_cuda = False
      sets.iters ='100,500,1000'
      sets.input_patch_size='136, 152, 136'
      sets.batch_size = 1 

      print("Current setting is:")
      print(sets)
      print("\n\n")

     
      print('sets.resume_path={}'.format(sets.resume_path))
      checkpoint = torch.load(sets.resume_path)
      net, _ = generate_model(sets)
      net = model=OuerNewModel(net,sets)
      net.load_state_dict(checkpoint['state_dict'])
      print('finished loading the model')

      
      testing_data = BratsSet(sets,sets.data_root, sets.img_list)
      data_loader = DataLoader(testing_data, batch_size=sets.batch_size, shuffle=False, num_workers=1, pin_memory=False)
      iter_nums = [ int(i) for i in sets.iters.split(",") ]
      with torch.no_grad():
              print('kupi')
              dices1,dices2,counter = test_calculate_metric(sets,iter_nums,net,data_loader)
              